#Загрузка данных


In [81]:
#загрузка и подключение библиотек
import pandas as pd        
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt 
import gensim.downloader as api   
import spacy 
import gensim.models
import string
from sklearn.metrics import confusion_matrix, classification_report  
from sklearn.linear_model import LogisticRegression  
from sklearn.model_selection import train_test_split
from gensim.models import FastText 

In [25]:
#загрузка датасета и проверка
data = pd.read_csv("spam_or_not_spam.csv")
data.head()

,email,label
0,date wed NUMBER aug NUMBER NUMBER NUMBER NUMB...,0
1,martin a posted tassos papadopoulos the greek ...,0
2,man threatens explosion in moscow thursday aug...,0
3,klez the virus that won t die already the most...,0
4,in adding cream to spaghetti carbonara which ...,0


In [4]:
#загрузка датасета слов
wordsdata = api.load('word2vec-google-news-300')

[==================================================] 100.0% 1662.8/1662.8MB downloaded


#Предобработка

In [26]:
#смена имён столбцов
data = data[['email','label']].rename(columns={'email':'text', 'label' :'spam'})
data.head()

,text,spam
0,date wed NUMBER aug NUMBER NUMBER NUMBER NUMB...,0
1,martin a posted tassos papadopoulos the greek ...,0
2,man threatens explosion in moscow thursday aug...,0
3,klez the virus that won t die already the most...,0
4,in adding cream to spaghetti carbonara which ...,0


In [31]:
#Слова-исключения + перевод в формат str
data['text']= data['text'].astype(str)
stop = spacy.load("en_core_web_sm")      
sw = stop.Defaults.stop_words   
' '.join(sw)      

"sometime to others two front ’m beside 'd one among both namely everything nothing up yours anyway toward several do am himself along just become there at beyond seemed a whatever afterwards no together towards above after eight behind with been 'll themselves say re wherever serious upon full of has was amount anything becoming through except whether also alone across whose became any first seeming my her whom formerly fifty next even three various than where otherwise own here name why another on your perhaps very until around before made always because see this therefore about give could noone does their thereafter really must besides take when may ’re enough most used our during whither once top never least whoever quite the meanwhile 're or everyone ’s much what former neither itself moreover me amongst sixty and still anyone however four without mine onto down did less that since nevertheless have few bottom nine will had should well whenever ’ve thereby thus forty something eac

In [32]:
#приведение текста в порядок
%%time
data['cleaned_text_sm'] = data['text'].apply(
    lambda x: ' '.join(
        token.lemma_.lower() for token in stop(x) if 
        not token.is_stop                             
        and not token.is_punct                        
        and not token.is_space                  
    )
)
data.sample(5)

CPU times: user 2min 35s, sys: 1.29 s, total: 2min 36s
Wall time: 2min 41s


,text,spam,cleaned_text_sm
948,from valdis kletnieks URL date mon NUMBER aug...,0,valdis kletnieks url date mon number aug numbe...
1008,on fri sep NUMBER NUMBER at NUMBER NUMBER robe...,0,fri sep number number number number robert elz...
625,on wed NUMBER sep NUMBER tom wrote the others ...,0,d number sep number tom write mailing list sta...
2247,url URL date NUMBER NUMBER NUMBERtNUMBER NUMBE...,0,url url date number number numbertnumber numbe...
1219,once upon a time brian wrote yeah i need to wo...,0,time brian write yeah need work learn lesson l...


#Gensim

In [34]:
#вектор слова "игра"
type(wordsdata)
word_vector = wordsdata['game']
word_vector.shape

(300,)

In [35]:
#близкие к словам "приложение" и "мобильный"
wordsdata.most_similar(positive=['app', 'mobile'], topn=10)

[('apps', 0.744373083114624),
 ('smartphone', 0.7307726144790649),
 ('iPhone', 0.6870308518409729),
 ('iPhone_app', 0.669269859790802),
 ('smartphones', 0.6636013388633728),
 ('web2go', 0.663374125957489),
 ('LifeInPocket', 0.663284182548523),
 ('Maps_Navigation', 0.6571023464202881),
 ('Android', 0.655826985836029),
 ('Android_apps', 0.6556190252304077)]

In [37]:
#Английский, но не язык
wordsdata.most_similar(positive=['english'], negative=['language'], topn=10)

[('england', 0.5057789087295532),
 ('morgan', 0.48939579725265503),
 ('eddie', 0.4889262914657593),
 ('birmingham', 0.4798719584941864),
 ('lewis', 0.47613799571990967),
 ('europe', 0.47424569725990295),
 ('clark', 0.4730987548828125),
 ('portsmouth', 0.47295787930488586),
 ('ricky', 0.46157118678092957),
 ('todd', 0.4607299566268921)]

In [39]:
#Проверка - слово "ресторан" должно быть дальше всех от бассейна, стадиона и трех типов слов "поле"
print(wordsdata.doesnt_match(['restaurant', 'pitch', 'field', 'stadium', 'pool']))

restaurant


In [44]:
#доп. обработка
data = data[data.text.str.len() != 0]
res = []
for doc in stop.pipe(data['text']):
    res.append([token.lemma_ for token in doc if token.lemma_ not in string.punctuation and not token.is_stop])

In [45]:
print(res[0])

[' ', 'date', 'd', 'NUMBER', 'aug', 'NUMBER', 'NUMBER', 'NUMBER', 'number', 'number', 'chris', 'garrigues', 'cwg', 'date', 'NUMBER', 'NUMBERfaNUMBERd', 'deepeddy', 'com', 'message', 'd', 'NUMBER', 'NUMBER', 'tmda', 'deepeddy', 'vircio', 'com', 't', 'reproduce', 'error', 'repeatable', 'like', 'time', 'fail', 'debug', 'log', 'pick', 'happen', 'NUMBER', 'NUMBER', 'NUMBER', 'pick_it', 'exec', 'pick', 'inbox', 'list', 'lbrace', 'lbrace', 'subject', 'ftp', 'rbrace', 'rbrace', 'number', 'number', 'sequence', 'mercury', 'NUMBER', 'NUMBER', 'NUMBER', 'exec', 'pick', 'inbox', 'list', 'lbrace', 'lbrace', 'subject', 'ftp', 'rbrace', 'rbrace', 'number', 'number', 'sequence', 'mercury', 'NUMBER', 'NUMBER', 'NUMBER', 'ftoc_pickmsgs', 'number', 'hit', 'NUMBER', 'number', 'number', 'mark', 'NUMBER', 'hit', 'NUMBER', 'NUMBER', 'NUMBER', 'tkerror', 'syntax', 'error', 'expression', 'int', 'note', 'run', 'pick', 'command', 'hand', 'delta', 'pick', 'inbox', 'list', 'lbrace', 'lbrace', 'subject', 'ftp', 'rbr

#SkipGram

In [46]:
%%time
model_W2V_skip_gram = gensim.models.Word2Vec(
    sentences=res, 
    vector_size=256, # vector_size, default = 100
    window=5, # default = 5
    min_count=3, 
    sg=1, # Training algorithm: 1 for skip-gram; otherwise CBOW
    hs=0, #  If 1, hierarchical softmax will be used for model training. If 0, and negative is non-zero, negative sampling will be used.
    negative=5, # If > 0, negative sampling will be used, if set to 0, no negative sampling is used.
    epochs=25, # epochs, Number of iterations (epochs) over the corpus
    seed=2023,
)

CPU times: user 2min 7s, sys: 193 ms, total: 2min 7s
Wall time: 1min 15s


In [52]:
#Для проверки взято слово "язык"
model_W2V_skip_gram.wv.most_similar(positive=['language'], topn=10) 

[('programming', 0.6503124833106995),
 ('elegance', 0.4966716170310974),
 ('irrespective', 0.4906229078769684),
 ('programmer', 0.47094032168388367),
 ('caller', 0.4517610967159271),
 ('namibian', 0.44651034474372864),
 ('commoditize', 0.4431207776069641),
 ('symbolize', 0.44183868169784546),
 ('fortran', 0.44070205092430115),
 ('nominally', 0.4396805763244629)]

In [53]:
#Взято слово из списка выше, и видно, что встречается слово "language"
model_W2V_skip_gram.wv.most_similar(positive=['programmer'], topn=10)

[('delphi', 0.5993934869766235),
 ('fortran', 0.5176477432250977),
 ('deficiency', 0.4902195632457733),
 ('mfc', 0.47235363721847534),
 ('language', 0.47094032168388367),
 ('symbolize', 0.4704573154449463),
 ('elegance', 0.46380260586738586),
 ('scripting', 0.4616515636444092),
 ('vc', 0.45758554339408875),
 ('alot', 0.4515017867088318)]

In [56]:
#Проверка - слово "ресторан" должно быть дальше всех слов из списка выше
model_W2V_skip_gram.wv.doesnt_match(['restaurant', 'symbolize', 'programmer', 'language', 'programming'])

'restaurant'

In [67]:
#Набор слов
wordsSG = model_W2V_skip_gram.wv.key_to_index.keys()

In [68]:
#Работа с векторами
i = 0
X_skip_gram = [] 
Y_skip_gram = []
for sentence in res:    
  veclist = []   
  for word in sentence: 
    if word in wordsSG: 
      veclist.append(model_W2V_skip_gram.wv[word])    
  if veclist:   
    X_skip_gram.append(np.mean(veclist, axis=0))                                                    
    Y_skip_gram.append(data.spam[i])  
  i = i + 1     

In [69]:
#Тренировка
X_train_skip_gram, X_test_skip_gram, y_train_skip_gram, y_test_skip_gram = train_test_split(X_skip_gram, Y_skip_gram, random_state=2303)

In [71]:
#Результат
sgReg = LogisticRegression()
sgReg.fit(X_train_skip_gram, y_train_skip_gram)
sgResult = sgReg.predict(X_test_skip_gram)
print(classification_report(y_test_skip_gram, sgResult))

              precision    recall  f1-score   support

           0       0.98      0.99      0.99       622
           1       0.95      0.92      0.94       128

    accuracy                           0.98       750
   macro avg       0.97      0.96      0.96       750
weighted avg       0.98      0.98      0.98       750



#CBOW

In [72]:
%%time
model_W2V_CBOW = gensim.models.Word2Vec(
    sentences=res, 
    vector_size=256, # vector_size, default = 100
    window=10, # default = 5
    min_count=3, 
    sg=0, # Training algorithm: 1 for skip-gram; otherwise CBOW
    hs=0, #  If 1, hierarchical softmax will be used for model training. If 0, and negative is non-zero, negative sampling will be used.
    negative=5, # If > 0, negative sampling will be used, if set to 0, no negative sampling is used.
    epochs=25, # epochs, Number of iterations (epochs) over the corpus
    seed=2023,
)

CPU times: user 42.3 s, sys: 146 ms, total: 42.4 s
Wall time: 26.4 s


In [73]:
#Для проверки взято слово "язык"
model_W2V_CBOW.wv.most_similar(positive=['language'], topn=10) 

[('programming', 0.8161123991012573),
 ('elegance', 0.739441990852356),
 ('programmer', 0.6886636018753052),
 ('java', 0.6446874737739563),
 ('scripting', 0.6305705308914185),
 ('delphi', 0.6244141459465027),
 ('design', 0.6235035061836243),
 ('fortran', 0.6217892169952393),
 ('billington', 0.6051723957061768),
 ('symbolize', 0.5915306806564331)]

In [74]:
#Взято слово из списка выше, и видно, что встречается слово "language"
model_W2V_CBOW.wv.most_similar(positive=['programmer'], topn=10)

[('programming', 0.7309220433235168),
 ('delphi', 0.7098283171653748),
 ('fortran', 0.6923387050628662),
 ('language', 0.6886637210845947),
 ('java', 0.6693075895309448),
 ('elegance', 0.628521203994751),
 ('design', 0.6279503703117371),
 ('imho', 0.5815724730491638),
 ('c', 0.5812642574310303),
 ('scripting', 0.5771560072898865)]

In [75]:
#Проверка - слово "ресторан" должно быть дальше всех слов из списка выше
model_W2V_CBOW.wv.doesnt_match(['restaurant', 'symbolize', 'programmer', 'language', 'programming'])

'restaurant'

In [77]:
#Набор слов
wordsSBOW = model_W2V_skip_gram.wv.key_to_index.keys() 

In [78]:
#Работа с векторами
i = 0
X_CBOW = [] 
Y_CBOW = []
for sentence in res:    
  veclist = []   
  for word in sentence:
    if word in wordsSBOW:   
      veclist.append(model_W2V_CBOW.wv[word])     
  if veclist:   
    X_CBOW.append(np.mean(veclist, axis=0))                                               
    Y_CBOW.append(data.spam[i])    
  i = i + 1     

In [80]:
#Тренировка и результат
X_train_CBOW, X_test_CBOW, y_train_CBOW, y_test_CBOW = train_test_split(X_CBOW, Y_CBOW, random_state=2303)
CBOWreg = LogisticRegression()   
CBOWreg.fit(X_train_CBOW, y_train_CBOW)   
CBOWresult = CBOWreg.predict(X_test_CBOW)   
print(classification_report(y_test_CBOW, CBOWresult))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99       622
           1       0.93      0.96      0.95       128

    accuracy                           0.98       750
   macro avg       0.96      0.97      0.97       750
weighted avg       0.98      0.98      0.98       750



#FastText

In [97]:
FTmodel = FastText(vector_size=256, window=10, min_count=5)
FTmodel.build_vocab(corpus_iterable = res)

In [98]:
%%time
FTmodel.train(corpus_iterable=res, total_examples=len(res), epochs=25)

CPU times: user 5min 51s, sys: 411 ms, total: 5min 51s
Wall time: 3min 28s


(8058834, 9970750)

In [106]:
#Для проверки взято слово "язык"
FTmodel.wv.most_similar(positive=['language'], topn=10) 

[('langur', 0.7923566102981567),
 ('java', 0.5576001405715942),
 ('lang', 0.5270918011665344),
 ('difficult', 0.5055602788925171),
 ('efficiency', 0.5022487044334412),
 ('scientific', 0.5004261136054993),
 ('programming', 0.496028333902359),
 ('difference', 0.48711633682250977),
 ('difficulty', 0.48566552996635437),
 ('programmer', 0.4815993905067444)]

In [107]:
#Взято слово из списка выше, и видно, что встречается слово "language"
FTmodel.wv.most_similar(positive=['programmer'], topn=10)

[('programme', 0.9576631784439087),
 ('program', 0.899738609790802),
 ('reprogramme', 0.895237147808075),
 ('programming', 0.8949148058891296),
 ('progressive', 0.7535768747329712),
 ('progress', 0.7083593010902405),
 ('gram', 0.6808252930641174),
 ('proxy', 0.6798794269561768),
 ('pro', 0.6483399868011475),
 ('proud', 0.6358023881912231)]

In [109]:
#Проверка - слово "ресторан" должно быть дальше всех слов из списка выше
FTmodel.wv.doesnt_match(['restaurant', 'scientific', 'programmer', 'language', 'programming'])

'restaurant'

In [110]:
#Набор слов
wordsFT = FTmodel.wv.key_to_index.keys()

In [111]:
#Работа с векторами
i = 0
X_fast_text = []  
Y_fast_text = []
for sentence in res:
  veclist = []
  for word in sentence: 
    if word in wordsFT:
      veclist.append(FTmodel.wv[word])   
  if veclist:
    X_fast_text.append(np.mean(veclist, axis=0))       
    Y_fast_text.append(data.spam[i])
  i = i + 1  

In [112]:
#Тренировка и результат
X_train_fast_text, X_test_fast_text, y_train_fast_text, y_test_fast_text = train_test_split(X_fast_text, Y_fast_text, random_state=2303)
FTreg = LogisticRegression()
FTreg.fit(X_train_fast_text, y_train_fast_text)
FTresult = FTreg.predict(X_test_fast_text)
print(classification_report(y_test_fast_text, FTresult))

              precision    recall  f1-score   support

           0       0.99      0.98      0.99       622
           1       0.90      0.97      0.93       128

    accuracy                           0.98       750
   macro avg       0.95      0.97      0.96       750
weighted avg       0.98      0.98      0.98       750



/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
